# ddt and bct

In [ ]:
# Saturnin 1->1 DDT (single-bit input diffs vs single-bit output diffs)
# S-boxes from Saturnin spec:
s0 = [0, 6,14, 1,15, 4, 7,13, 9, 8,12, 5, 2,10, 3,11]
s1 = [0, 9,13, 2,15, 1,11, 7, 6, 4, 5, 3, 8,12,10,14]

def bit_positions(nbits=4):
    return [1 << i for i in range(nbits)]

def compute_1to1_ddt(sbox):
    """
    Returns a 4x4 matrix M where
      M[i][j] = number of x in 0..15 s.t.
                sbox[x] ^ sbox[x ^ in_diff] == out_diff
    where in_diff = 1<<i and out_diff = 1<<j (i,j in 0..3).
    """
    nbits = 4
    in_diffs = bit_positions(nbits)   # [1,2,4,8]
    out_diffs = bit_positions(nbits)
    M = [[0]*nbits for _ in range(nbits)]
    for i, ind in enumerate(in_diffs):
        for j, outd in enumerate(out_diffs):
            cnt = 0
            for x in range(16):
                y = x ^ ind
                if (sbox[x] ^ sbox[y]) == outd:
                    cnt += 1
            M[i][j] = cnt
    return M

def pretty_print_matrix(M, title):
    print(title)
    print("input-bit-diff -> output-bit-diff")
    header = " in\\out | " + " ".join(f"{1<<j:2d}" for j in range(4))
    print(header)
    print("-" * len(header))
    for i,row in enumerate(M):
        print(f"  {1<<i:2d}    | " + " ".join(f"{v:2d}" for v in row))
    print()

if __name__ == "__main__":
    M0 = compute_1to1_ddt(s0)
    M1 = compute_1to1_ddt(s1)
    pretty_print_matrix(M0, "σ0  (Saturnin) 1→1 DDT (rows: input single-bit diffs, cols: output single-bit diffs)")
    pretty_print_matrix(M1, "σ1  (Saturnin) 1→1 DDT (rows: input single-bit diffs, cols: output single-bit diffs)")


σ0  (Saturnin) 1→1 DDT (rows: input single-bit diffs, cols: output single-bit diffs)
input-bit-diff -> output-bit-diff
 in\out |  1  2  4  8
---------------------
   1    |  2  0  0  4
   2    |  4  0  0  2
   4    |  0  4  0  0
   8    |  0  2  4  0

σ1  (Saturnin) 1→1 DDT (rows: input single-bit diffs, cols: output single-bit diffs)
input-bit-diff -> output-bit-diff
 in\out |  1  2  4  8
---------------------
   1    |  0  2  4  0
   2    |  0  4  2  0
   4    |  0  0  0  4
   8    |  4  0  0  2



In [3]:
# ✅ Full DDT (Differential Distribution Table) for Saturnin σ0 and σ1

# Saturnin S-boxes (4-bit → 4-bit)
sigma0 = [0, 6,14, 1,15, 4, 7,13, 9, 8,12, 5, 2,10, 3,11]
sigma1 = [0, 9,13, 2,15, 1,11, 7, 6, 4, 5, 3, 8,12,10,14]

def compute_ddt(S):
    """Computes full 16x16 DDT of a 4-bit S-box."""
    n = len(S)
    ddt = [[0]*n for _ in range(n)]
    for a in range(n):        # input difference
        for x in range(n):
            y = x ^ a
            b = S[x] ^ S[y]   # output difference
            ddt[a][b] += 1
    return ddt

def print_ddt(ddt, name):
    print(f"\nDDT of {name}:")
    print("a\\b | " + " ".join(f"{i:2d}" for i in range(16)))
    print("-"*45)
    for a, row in enumerate(ddt):
        print(f"{a:2d} | " + " ".join(f"{v:2d}" for v in row))

# Compute DDTs
ddt_s0 = compute_ddt(sigma0)
ddt_s1 = compute_ddt(sigma1)

# Display
print_ddt(ddt_s0, "σ0 (Saturnin)")
print_ddt(ddt_s1, "σ1 (Saturnin)")



DDT of σ0 (Saturnin):
a\b |  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
---------------------------------------------
 0 | 16  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1 |  0  2  0  0  0  0  2  0  4  2  2  2  0  0  0  2
 2 |  0  4  0  0  0  2  0  2  2  2  0  0  0  2  2  0
 3 |  0  2  2  2  2  0  0  0  2  4  0  0  2  0  0  0
 4 |  0  0  4  0  0  0  0  0  0  2  0  2  2  0  2  4
 5 |  0  0  0  4  2  0  4  2  0  2  2  0  0  0  0  0
 6 |  0  2  0  2  0  2  0  2  0  0  2  2  0  0  2  2
 7 |  0  2  2  0  0  0  2  2  0  0  2  2  0  2  2  0
 8 |  0  0  2  0  4  0  2  0  0  2  0  0  0  2  4  0
 9 |  0  0  0  0  0  2  2  0  2  0  0  2  2  2  2  2
10 |  0  0  0  2  0  2  0  4  0  2  0  0  4  0  0  2
11 |  0  0  0  0  2  2  2  2  2  0  2  0  0  2  0  2
12 |  0  0  2  0  0  0  2  0  2  0  2  2  4  2  0  0
13 |  0  2  4  0  2  2  0  2  0  0  2  0  0  2  0  0
14 |  0  2  0  4  0  2  0  0  0  0  0  2  2  2  2  0
15 |  0  0  0  2  4  2  0  0  2  0  2  2  0  0  0  2

DDT of σ1 (Saturnin):
a\b | 

In [5]:
# ✅ Boomerang Connectivity Table (BCT) for Saturnin
# Definition: BCT[a][b] = |{ x | S⁻¹(S(x)⊕b) ⊕ S⁻¹(S(x⊕a)⊕b) = a }|

sigma0 = [0, 6,14, 1,15, 4, 7,13, 9, 8,12, 5, 2,10, 3,11]
sigma1 = [0, 9,13, 2,15, 1,11, 7, 6, 4, 5, 3, 8,12,10,14]

def invert_sbox(S):
    """Return inverse permutation of a 4-bit S-box."""
    inv = [0]*len(S)
    for i, v in enumerate(S):
        inv[v] = i
    return inv

def compute_bct(S):
    """Compute 16x16 Boomerang Connectivity Table for 4-bit S-box S."""
    n = len(S)
    invS = invert_sbox(S)
    bct = [[0]*n for _ in range(n)]

    for a in range(n):
        for b in range(n):
            count = 0
            for x in range(n):
                left  = invS[S[x] ^ b]
                right = invS[S[x ^ a] ^ b]
                if left ^ right == a:
                    count += 1
            bct[a][b] = count
    return bct

def print_bct(bct, name):
    print(f"\nBCT of {name}:")
    print("a\\b | " + " ".join(f"{i:2d}" for i in range(16)))
    print("-"*45)
    for a, row in enumerate(bct):
        print(f"{a:2d} | " + " ".join(f"{v:2d}" for v in row))

# Compute BCTs
bct_s0 = compute_bct(sigma0)
bct_s1 = compute_bct(sigma1)

# Display
print_bct(bct_s0, "σ0 (Saturnin)")
print_bct(bct_s1, "σ1 (Saturnin)")



BCT of σ0 (Saturnin):
a\b |  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
---------------------------------------------
 0 | 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16
 1 | 16  6  0  0  0  0  2  0  4  6  2  2  0  0  0  2
 2 | 16  4  0  0  0  2  0  2  6  6  0  0  0  2  2  0
 3 | 16  6  2  2  2  0  0  0  6  4  0  0  2  0  0  0
 4 | 16  0  4  4  0  0  0  0  0  2  0  2 10  0  6  4
 5 | 16  0  4  4 10  0  4  6  0  2  2  0  0  0  0  0
 6 | 16  2  0  2  0  2  0  2  0  0  2  2  0  0  2  2
 7 | 16  2  2  0  0  0  2  2  0  0  2  2  0  2  2  0
 8 | 16  0 10  0  4  0  6  0  0  2  0  0  4  2  4  0
 9 | 16  0  0  0  0  2  2  0  2  0  0  2  2  2  2  2
10 | 16  0  0 10  4  2  0  4  0  2  0  0  4  0  0  6
11 | 16  0  0  0  2  2  2  2  2  0  2  0  0  2  0  2
12 | 16  0  6  0  0  0  2  0  2  0  2  2  4  2  4  0
13 | 16  2  4  0  6  2  4  2  0  0  2  0  0  2  0  0
14 | 16  2  0  4  0  2  0  0  0  0  0  2  6  2  2  4
15 | 16  0  0  6  4  2  0  4  2  0  2  2  0  0  0  2

BCT of σ1 (Saturnin):
a\b | 

# analysis

In [7]:
def differential_uniformity(ddt):
    """Compute differential uniformity = max DDT[a][b] for a ≠ 0."""
    max_val = 0
    pairs = []
    for a in range(1, len(ddt)):       # exclude a = 0
        for b in range(len(ddt)):
            if ddt[a][b] > max_val:
                max_val = ddt[a][b]
                pairs = [(a, b)]
            elif ddt[a][b] == max_val:
                pairs.append((a, b))
    return max_val, pairs

def spectral_distribution(ddt):
    """Return histogram: how many (a,b) pairs give 0,2,4,..."""
    hist = {}
    for a in range(len(ddt)):
        for b in range(len(ddt)):
            val = ddt[a][b]
            hist[val] = hist.get(val, 0) + 1
    return dict(sorted(hist.items()))  # sorted by value

# --- Run analysis for both σ0 and σ1 ---
ddt_s0 = compute_ddt(sigma0)
ddt_s1 = compute_ddt(sigma1)

du_s0, pairs_s0 = differential_uniformity(ddt_s0)
du_s1, pairs_s1 = differential_uniformity(ddt_s1)

spec_s0 = spectral_distribution(ddt_s0)
spec_s1 = spectral_distribution(ddt_s1)

print("=== Differential Uniformity ===")
print(f"σ0: DU = {du_s0}, at (a,b) = {pairs_s0}")
print(f"σ1: DU = {du_s1}, at (a,b) = {pairs_s1}")

print("\n=== Spectral Distribution (value : count) ===")
print("σ0:", spec_s0)
print("σ1:", spec_s1)

=== Differential Uniformity ===
σ0: DU = 4, at (a,b) = [(1, 8), (2, 1), (3, 9), (4, 2), (4, 15), (5, 3), (5, 6), (8, 4), (8, 14), (10, 7), (10, 12), (12, 12), (13, 2), (14, 3), (15, 4)]
σ1: DU = 4, at (a,b) = [(1, 4), (2, 2), (3, 6), (4, 8), (4, 15), (5, 9), (5, 10), (8, 1), (8, 13), (10, 5), (10, 11), (12, 5), (13, 8), (14, 10), (15, 1)]

=== Spectral Distribution (value : count) ===
σ0: {0: 150, 2: 90, 4: 15, 16: 1}
σ1: {0: 150, 2: 90, 4: 15, 16: 1}


In [10]:
def boomerang_uniformity(bct):
    max_val = 0
    for a in range(len(bct)):
        for b in range(len(bct)):
            if a == 0 or b == 0:
                continue
            max_val = max(max_val, bct[a][b])
    return max_val

# Compute boomerang uniformity
bu0 = boomerang_uniformity(bct_s0)
bu1 = boomerang_uniformity(bct_s1)

print("Boomerang Uniformity of σ0:", bu0)
print("Boomerang Uniformity of σ1:", bu1)

Boomerang Uniformity of σ0: 10
Boomerang Uniformity of σ1: 10


# bogi

### saturnin is having all 11DDT inputs as bad input and all outputs as bad outputs

In [1]:
# 4-bit S-box
s0 = [0, 6, 14, 1, 15, 4, 7, 13, 9, 8, 12, 5, 2, 10, 3, 11]

n = 4
SIZE = 1 << n  # 16


def parity(x):
    """Return parity (sum of bits mod 2)."""
    return bin(x).count("1") & 1


def inner_product(a, b):
    """Compute binary inner product <a,b> over F2^n."""
    return parity(a & b)


def dlct_definition5(sbox):
    """Compute DLCT table using Definition 5 from your screenshot."""
    table = [[0 for _ in range(SIZE)] for _ in range(SIZE)]

    for delta_i in range(SIZE):        # input difference
        for lambda_o in range(SIZE):   # output mask
            dlct0 = 0
            dlct1 = 0
            for x in range(SIZE):
                val = inner_product(lambda_o, sbox[x]) ^ inner_product(lambda_o, sbox[x ^ delta_i])
                if val == 0:
                    dlct0 += 1
                else:
                    dlct1 += 1
            table[delta_i][lambda_o] = abs(dlct0) - abs(dlct1)
    return table


dlct = dlct_definition5(s0)

print("DLCT (Definition 5 form):")
for delta_i in range(SIZE):
    row = " ".join(f"{dlct[delta_i][lambda_o]:3d}" for lambda_o in range(SIZE))
    print(f"Δi={delta_i:2d}: {row}")


# Optional: normalized DLCT
normalized_dlct = [[dlct[a][b] / SIZE for b in range(SIZE)] for a in range(SIZE)]


DLCT (Definition 5 form):
Δi= 0:  16  16  16  16  16  16  16  16  16  16  16  16  16  16  16  16
Δi= 1:  16   0   0   0   8   0   8   0  -8   0   0  -8  -8  -8   0   0
Δi= 2:  16  -8   8  -8   0   0   8   0   0  -8   0   0   0   0   0  -8
Δi= 3:  16   0   8   0   8  -8   0  -8   0   0  -8   0   0   0  -8   0
Δi= 4:  16   0  -8   0   0   0   0  -8  -8   8   0  -8   8   8  -8   0
Δi= 5:  16   0  -8   0   0  -8   0   0   8   0  -8   8  -8  -8   0   8
Δi= 6:  16  -8  -8   0   0   0   0   0   0  -8   8   0   0   0   0   0
Δi= 7:  16   0  -8  -8   0   0   0   0   0   0   0   0   0   0   8  -8
Δi= 8:  16   8   0  -8  -8  -8   0   0   0   8   0   8   0   0  -8  -8
Δi= 9:  16   0   0   0  -8   0   0   8  -8   0   0  -8   0   0   0   0
Δi=10:  16  -8   0   8  -8   0   0  -8   0  -8  -8   0   0   8   0   8
Δi=11:  16   0   0   0  -8   8   0   0   0   0   0   0  -8  -8   0   0
Δi=12:  16   8   0   0   0   0  -8   0  -8   0  -8  -8   0   0   8   0
Δi=13:  16   0   0  -8   0   8  -8  -8   8   0   0 